In [13]:
# The data given in the url is related with direct marketing campaigns of a banking
# institution. The marketing campaigns were based on phone calls. Often, more
# than one contact to the same client was required, in order to access if the product
# (bank term deposit) would be ('yes') or not ('no') subscribed. Build a model to
# predict whether client will subscribe to term deposit

import pandas as pd
import numpy as np

In [14]:
data = pd.read_csv("bank.csv",sep=";")
data.head()

,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,no,1787,no,no,cellular,19,oct,79,1,-1,0,unknown,no
1,33,services,married,secondary,no,4789,yes,yes,cellular,11,may,220,1,339,4,failure,no
2,35,management,single,tertiary,no,1350,yes,no,cellular,16,apr,185,1,330,1,failure,no
3,30,management,married,tertiary,no,1476,yes,yes,unknown,3,jun,199,4,-1,0,unknown,no
4,59,blue-collar,married,secondary,no,0,yes,no,unknown,5,may,226,1,-1,0,unknown,no


In [15]:
data.replace(['yes','no'],[1,0],inplace=True)
data.head()


,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,30,unemployed,married,primary,0,1787,0,0,cellular,19,oct,79,1,-1,0,unknown,0
1,33,services,married,secondary,0,4789,1,1,cellular,11,may,220,1,339,4,failure,0
2,35,management,single,tertiary,0,1350,1,0,cellular,16,apr,185,1,330,1,failure,0
3,30,management,married,tertiary,0,1476,1,1,unknown,3,jun,199,4,-1,0,unknown,0
4,59,blue-collar,married,secondary,0,0,1,0,unknown,5,may,226,1,-1,0,unknown,0


In [16]:
#convert categorical data
data=pd.get_dummies(data)
data.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,30,0,1787,0,0,19,79,1,-1,0,...,0,0,0,0,1,0,0,0,0,1
1,33,0,4789,1,1,11,220,1,339,4,...,0,0,1,0,0,0,1,0,0,0
2,35,0,1350,1,0,16,185,1,330,1,...,0,0,0,0,0,0,1,0,0,0
3,30,0,1476,1,1,3,199,4,-1,0,...,1,0,0,0,0,0,0,0,0,1
4,59,0,0,1,0,5,226,1,-1,0,...,0,0,1,0,0,0,0,0,0,1


In [17]:
def feature_scaling(data):
    for i in data.columns:
        data[i]=(data[i]-data[i].min())/(data[i].max()-data[i].min())
    return data
data=feature_scaling(data)
data.head()

,age,default,balance,housing,loan,day,duration,campaign,pdays,previous,...,month_jun,month_mar,month_may,month_nov,month_oct,month_sep,poutcome_failure,poutcome_other,poutcome_success,poutcome_unknown
0,0.161765,0.0,0.068455,0.0,0.0,0.600000,0.024826,0.000000,0.000000,0.00,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0
1,0.205882,0.0,0.108750,1.0,1.0,0.333333,0.071500,0.000000,0.389908,0.16,...,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,0.235294,0.0,0.062590,1.0,0.0,0.500000,0.059914,0.000000,0.379587,0.04,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.161765,0.0,0.064281,1.0,1.0,0.066667,0.064548,0.061224,0.000000,0.00,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,0.588235,0.0,0.044469,1.0,0.0,0.133333,0.073486,0.000000,0.000000,0.00,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [18]:
def split_train_test(data):
    train=len(data)*0.70
    train=int(train)
    test=len(data)-train
    test=int(test)
    
    train=data.head(train)
    test=data.tail(test)
    
    return train,test
train,test=split_train_test(data)

def separate(data):
    output=data.y
    return data.drop('y',axis=1),output
x_train,y_train=separate(train)
x_test,y_test=separate(test)

In [27]:
class MultiNeuralNetwork:
    def __init__(self):
        self.learning_rate=0.1
        self.epoch=1000
        
    def train(self,x_train_data,y_train_data):
        layers=[x_train_data.shape[1],4,3,1]
        print("layers",layers)
        A=[0]*len(layers)
        A[0]=x_train_data.T
        z=[0]*len(layers)
        dg=[0]*len(layers)
        da=[0]*len(layers)
        dz=[0]*len(layers)
        dw=[0]*len(layers)
        db=[0]*len(layers)
        da1=[0]*len(layers)
        da2=[0]*len(layers)
        weight=[]
        bias=[]
        # Initializing weights and bias for all layers 
        for i in range(len(layers)):
            weight.append((np.random.randn(layers[i],layers[i-1])*0.1))
            bias.append(np.zeros((layers[i],1)))
        
        for data in range(self.epoch):
        #forward propogation
            for i in range(1,len(layers)):
                #hypothesis function
                z[i]=np.dot(weight[i],A[i-1])+bias[i]
                # Activation function
                A[i]=np.divide(1,np.add(1,np.exp(-z[i])))

        #backward propogation
            for j in reversed(range(1,len(layers))):
                #derivative of sigmoid
                dg[j]=np.multiply(A[j],(1-A[j]))
                
                da1[j]=np.divide(-y_train_data[j],A[j])#           
                da2[j]=np.divide((1-y_train_data[j]),(1-A[j]))
               
                da[j]=np.add(da1[j],da2[j])
                
                dz[j]=np.multiply(dg[j],da[j])
                
                dw[j]=np.dot(dz[j],A[j-1].T)/len(x_train_data)
                
                weight[j]=weight[j]-np.dot(self.learning_rate,dw[j])
                
                bias[j]=bias[j]-np.dot(self.learning_rate,db[j])
                
        return weight,bias
    def accuracy(self,y_test_data,y_predict):
        accuracy=100-(np.mean(np.abs(y_predict-y_test_data))*100)
        return accuracy
    
    def test(self,x_test_data,weight,bias):
        layers=[x_test_data.shape[1],4,3,1]
        z=[0]*len(layers)
        A=[0]*len(layers)
        A[0]=x_test_data.T
        for i in range(1,len(layers)):
            #hypothesis function
            z[i]=np.dot(weight[i],A[i-1])+bias[i]
 
            # Sigmoid Activation function
            A[i]=np.divide(1,np.add(1,np.exp(-z[i])))
        return A[-1]


In [28]:
def main():
    obj=MultiNeuralNetwork()
    
    x_train_data=np.array(x_train)
#    print("x train data",x_train_data.shape)
    y_train_data=np.array(y_train)
    y_train_data=y_train_data.reshape(len(y_train_data),1)
#    print("y_train_data shape",y_train_data.shape)
    
    
    x_test_data=np.array(x_test)
#    print("x test data",x_test_data.shape)
    y_test_data=np.array(y_test)
    y_test_data=y_test_data.reshape(len(y_test_data),1)
#    print("y_test_data",y_test_data.shape)
    

    weight,bias=obj.train(x_train_data,y_train_data)
    y_predict=obj.test(x_test_data,weight,bias)
    print("predication",y_predict)
    
    
    final_accuracy=obj.accuracy(y_test_data,y_predict)
    print("final accurancy",final_accuracy)
                      
if __name__=='__main__':
    main()

layers [48, 4, 3, 1]
predication [[0.01351435 0.01349459 0.01380793 ... 0.01350179 0.0149391  0.01435384]]
final accurancy 87.50578775610505
